# Elastic (Elastic Stack / Elasticsearch / Kibana / Elastic Security) — Defensive Playbook (Notebook)

This notebook is a **defender-focused reference** for using **Elastic** to ingest security telemetry, hunt threats, and automate response actions.

**Scope**
- Elastic Stack basics (components & security-relevant architecture)
- Data ingestion for security (Elastic Agent/Fleet, Beats, integrations)
- Query languages: **KQL**, **EQL**, **ES|QL**, and Query DSL
- Example detection/hunting queries for common attack categories (MITRE-style)
- Response actions (safe subset) and relevant APIs

> ⚠️ Safety note: This notebook focuses on **detection, investigation, and response**. It intentionally avoids offensive “how-to exploit” steps and avoids sharing endpoint “execute arbitrary commands” examples.


## 1) Elastic components (security view)

- **Elasticsearch**: stores and searches data (indices/data streams).
- **Kibana**: UI for search, dashboards, and **Elastic Security** (SIEM/XDR).
- **Elastic Agent + Fleet**: recommended ingestion/management model for endpoints and integrations.
- **Elastic Defend**: endpoint protection & telemetry (EDR) via Elastic Agent integration.
- **Logstash / Ingest pipelines**: parsing & enrichment.
- **ECS (Elastic Common Schema)**: standard fields that make correlation and detection easier.

Key security idea: Normalize everything into **ECS**, then write detections/hunts that rely on stable fields like:
- `@timestamp`, `event.category`, `event.type`, `event.action`, `event.outcome`
- `host.*`, `agent.*`, `user.*`, `process.*`, `file.*`, `network.*`, `source.*`, `destination.*`, `url.*`


## 2) What you need to defend well with Elastic

### Telemetry sources (typical)
- **Endpoints**: Windows Security logs + Sysmon, Linux auditd, macOS Unified Logs, EDR telemetry (Elastic Defend).
- **Network**: firewall, proxy, DNS, NetFlow, Zeek, IDS.
- **Identity**: AD/Entra ID, Okta, VPN, PAM.
- **Email**: M365, Google Workspace, SEG logs.
- **Cloud**: AWS CloudTrail, GuardDuty, VPC flow logs; Azure Activity; GCP audit logs.
- **Apps**: web server logs, WAF, IAM, CI/CD, SaaS audit logs.

### Elastic features defenders typically use
- **Detections engine** (rules → alerts)
- **Timeline** (investigation)
- **Cases** (workflow)
- **Dashboards** (visibility)
- **Threat intelligence** integrations (enrichment)
- **Response console** (containment & investigation actions)


## 3) Quick start: Elastic Agent (Fleet-managed) installation concepts

Fleet-managed Elastic Agent is generally recommended:
1. Set up **Fleet Server**
2. Create an **Agent policy** (include integrations like Elastic Defend, Windows, Sysmon, auditd, etc.)
3. Install the agent using an enrollment command generated by Kibana Fleet UI

The exact command is generated in your Kibana UI for your environment.


In [ ]:

# Example placeholders for an agent install flow (DO NOT run blindly).
# In Kibana: Management → Fleet → Add agent → choose your policy → copy the install command.
#
# Linux (tar) usually looks conceptually like:
#   sudo ./elastic-agent install --url=https://<fleet-server>:8220 --enrollment-token=<token>
#
# For your environment, copy the exact command from Kibana Fleet.
print("Use Kibana Fleet UI to generate the correct enrollment command for your environment.")


## 4) Elastic Agent CLI command reference (high-level)

Common Elastic Agent CLI concepts you’ll see:
- `install` (install as service)
- `enroll` (enroll into Fleet; sometimes combined with install depending on platform/package)
- `status`, `inspect`, `logs`
- `uninstall`

(See Elastic Agent command reference in Elastic docs.)


## 5) Elastic Endpoint (Elastic Defend) local troubleshooting commands

Elastic Endpoint (installed component for Elastic Defend) has its own CLI for **diagnostics and troubleshooting**.
Examples include:
- `elastic-endpoint status`
- `elastic-endpoint diagnostics`
- `elastic-endpoint inspect`
- `elastic-endpoint memorydump`

You must run with elevated privileges and include the full path because it is not on PATH by default.


In [ ]:

# Example paths (from Elastic docs):
# Windows: "C:\Program Files\Elastic\Endpoint\elastic-endpoint.exe"
# macOS:   /Library/Elastic/Endpoint/elastic-endpoint
# Linux:   /opt/Elastic/Endpoint/elastic-endpoint
#
# Example (Linux/macOS):
#   sudo /opt/Elastic/Endpoint/elastic-endpoint status --output json
#
# Example (Windows PowerShell as Administrator):
#   & "C:\Program Files\Elastic\Endpoint\elastic-endpoint.exe" status --output json


## 6) Response console (defensive) commands in Elastic Security

In Elastic Security → Response console, you can run response action commands against enrolled endpoints (requires appropriate privileges).
Common defensive actions include:
- `isolate` / `release` (containment)
- `status` (agent/policy/health)
- `processes` (list processes)
- `kill-process` / `suspend-process`
- `get-file` (retrieve a file for analysis)
- `scan` (scan a file/path)
- `memory-dump` (collect memory dump)

> ⚠️ Note: Some commands like `execute`, `upload`, and `runscript` can run arbitrary actions and are dual-use.
> This notebook **does not provide examples** for those. Prefer controlled, audited response workflows.


In [ ]:

# Example response console commands (safe subset)
# These are typed in Kibana → Elastic Security → Response console (not in a shell):

# isolate --comment "Contain host due to suspected compromise"
# status
# processes
# kill-process --pid <PID> --comment "Terminate confirmed malicious process"
# suspend-process --pid <PID> --comment "Suspend for live response"
# get-file --path "<absolute_path_to_file>" --comment "Collect artifact for analysis"
# scan --path "<path_or_file>" --comment "Scan suspected location"
# memory-dump --comment "Capture memory for IR"

print("These commands are run inside the Elastic Security response console UI.")


## 7) APIs defenders commonly use

### Elasticsearch APIs (direct to Elasticsearch)
- Authentication & authorization: `/_security/*`
- Search: `/<index>/_search`
- EQL: `/<index>/_eql/search`
- ES|QL: `/_query`
- ILM policies: `/_ilm/policy/*`
- Ingest pipelines: `/_ingest/pipeline/*`
- Index templates: `/_index_template/*`

### Kibana APIs (to manage detections/response actions via Kibana)
- Detections rules APIs (create/find/update rules)
- Endpoint isolate/unisolate actions

**Tip:** Use API keys and least-privilege roles.


## 8) Setup environment variables for API calls

You’ll typically need:
- `ELASTICSEARCH_URL` (e.g., https://your-es:9200)
- `KIBANA_URL` (e.g., https://your-kibana:5601)
- `ELASTIC_API_KEY` (base64 key string) or Bearer token


In [ ]:

# In a terminal (or a %%bash cell) you might set:
# export ELASTICSEARCH_URL="https://localhost:9200"
# export KIBANA_URL="https://localhost:5601"
# export ELASTIC_API_KEY="<base64_api_key>"

# In Python:
import os
os.environ.get("ELASTICSEARCH_URL", "<not set>"), os.environ.get("KIBANA_URL", "<not set>")


## 9) Minimal Elasticsearch API checks (curl)

Below are **defender-friendly** API calls to validate connectivity and get basic cluster/index health.


In [ ]:

%%bash
# NOTE: This cell is a template. Set env vars before running.
# Requires: curl

# echo "Elasticsearch URL: $ELASTICSEARCH_URL"
# curl -sS -H "Authorization: ApiKey $ELASTIC_API_KEY" "$ELASTICSEARCH_URL/"
# curl -sS -H "Authorization: ApiKey $ELASTIC_API_KEY" "$ELASTICSEARCH_URL/_cluster/health?pretty"
# curl -sS -H "Authorization: ApiKey $ELASTIC_API_KEY" "$ELASTICSEARCH_URL/_cat/indices?v"


## 10) Query languages overview

### KQL (Kibana Query Language)
- Used primarily in Kibana UI for filtering.
- Great for quick filtering; does **not** do transforms/aggregations itself.

### EQL (Event Query Language)
- Great for event sequences and correlations (e.g., "process A followed by network connection B").

### ES|QL (Elasticsearch Query Language)
- Piped query language for filtering, transforming, aggregating, and investigating in one query.

### Query DSL
- Elasticsearch JSON query language for full power (bool queries, aggregations, etc.).


## 11) KQL cheatsheet (security-relevant patterns)

KQL patterns you’ll use constantly:
- Existence: `field:*`
- Exact match: `user.name:"alice"`
- Terms list: `event.category:(authentication or process)`
- Wildcards: `process.name:("powershell*" or "cmd*")`
- Ranges: `event.duration > 1000000000`
- Negation: `not host.name:"known-good-host"`
- CIDR: `source.ip:10.0.0.0/8` (depending on mapping/type)


## 12) EQL cheatsheet (security-relevant patterns)

EQL is written per index/data stream and typically uses ECS fields:
- `process where process.name == "powershell.exe"`
- Sequences across categories:
  - `sequence by host.id with maxspan=5m [process ...] [network ...]`

You’ll need EQL-enabled indices and well-normalized event categories/types.


## 13) ES|QL cheatsheet (security-relevant patterns)

ES|QL uses a piped syntax:
- `FROM logs-* | WHERE event.category == "authentication" | LIMIT 50`
- Aggregations:
  - `... | STATS count = COUNT(*) BY user.name | SORT count DESC`
- Time windows often rely on filtering by `@timestamp`.


# 14) Detection & Hunting Patterns by Attack Category

Each section below includes:
- **What to collect**
- **Example KQL**
- **Example EQL** (when sequences help)
- **Example ES|QL** (when transforms/aggregations help)
- **Triage tips** (what to look at next)

> Field names assume ECS-style mappings (common in Elastic integrations).
> You must adapt queries to your exact data sources and mappings.


## 14.1 Password spraying / brute-force authentication

### What to collect
- Authentication logs (AD, Entra ID/Okta, VPN) normalized to ECS `event.category:authentication`
- Fields: `user.name`, `source.ip`, `event.outcome`, `event.action`/`event.type`, `related.user`, `related.ip`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:authentication and event.outcome:failure and user.name:* and source.ip:*


### Example EQL


In [ ]:
# EQL (use EQL search against the relevant index)
sequence by source.ip with maxspan=30m
  [ authentication where event.outcome == "failure" ]
  [ authentication where event.outcome == "success" ]


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "authentication" AND event.outcome == "failure"
| STATS failures = COUNT(*) BY user.name, source.ip
| WHERE failures >= 20
| SORT failures DESC
| LIMIT 100


### Triage tips
- Pivot on `source.ip` and check geo / ASN enrichment if available.
- Check if affected accounts share attributes (same group/role, same OU).
- Look for conditional access / MFA challenges and 'impossible travel' signals.


## 14.2 Credential stuffing (web/app logins)

### What to collect
- Web app auth logs (reverse proxy, WAF, app audit) with `url.path`, `http.*`, `user.name`/`user.email`
- Fields: `source.ip`, `user_agent.original`, `event.outcome`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:authentication and event.outcome:failure and url.path:(*login* or *signin* or *auth*) and source.ip:* and user_agent.original:*


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "authentication" AND event.outcome == "failure"
| STATS failures = COUNT(*), uas = VALUES(user_agent.original) BY source.ip, url.path
| WHERE failures >= 100
| SORT failures DESC
| LIMIT 50


### Triage tips
- Check if the same `source.ip` hits many different `user.name` values.
- Compare user agents: automation often reuses a small set.
- Look for spikes in 401/403 and WAF bot signals.


## 14.3 Suspicious PowerShell / scripting on endpoints

### What to collect
- Process creation telemetry (`event.category:process`) from Sysmon/EDR
- Command-line fields: `process.command_line`, `process.args`
- Parent/child process context: `process.parent.*`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:process and process.name:(powershell.exe or pwsh.exe) and process.command_line:(*EncodedCommand* or *FromBase64String* or *IEX* or *Invoke-Expression*)


### Example EQL


In [ ]:
# EQL (use EQL search against the relevant index)
sequence by host.id with maxspan=5m
  [ process where process.name == "powershell.exe" and wildcard(process.command_line, "*EncodedCommand*") ]
  [ network where true ]


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "process" AND process.name IN ("powershell.exe","pwsh.exe")
| WHERE process.command_line LIKE "*EncodedCommand*" OR process.command_line LIKE "*Invoke-Expression*" OR process.command_line LIKE "*FromBase64String*"
| KEEP @timestamp, host.name, user.name, process.parent.name, process.name, process.command_line
| SORT @timestamp DESC
| LIMIT 200


### Triage tips
- Review full command line and parent process (e.g., Office → PowerShell).
- Check outbound network connections shortly after execution.
- Look for file writes to temp directories and suspicious DLL loads.


## 14.4 Malicious Office macro / document chain

### What to collect
- Process telemetry for Office apps and child processes
- Fields: `process.parent.name`, `process.name`, `process.command_line`, `file.path`
- Email telemetry if available (attachments, sender, subject)


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:process and process.parent.name:(WINWORD.EXE or EXCEL.EXE or POWERPNT.EXE or OUTLOOK.EXE) and process.name:(powershell.exe or cmd.exe or wscript.exe or cscript.exe or rundll32.exe or regsvr32.exe or mshta.exe)


### Example EQL


In [ ]:
# EQL (use EQL search against the relevant index)
sequence by host.id with maxspan=10m
  [ process where process.parent.name in ("WINWORD.EXE","EXCEL.EXE","POWERPNT.EXE","OUTLOOK.EXE") ]
  [ process where process.name in ("powershell.exe","cmd.exe","wscript.exe","cscript.exe","rundll32.exe","mshta.exe") ]


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "process" AND process.parent.name IN ("WINWORD.EXE","EXCEL.EXE","POWERPNT.EXE","OUTLOOK.EXE")
| WHERE process.name IN ("powershell.exe","cmd.exe","wscript.exe","cscript.exe","rundll32.exe","regsvr32.exe","mshta.exe")
| KEEP @timestamp, host.name, user.name, process.parent.name, process.name, process.command_line
| SORT @timestamp DESC
| LIMIT 200


### Triage tips
- Confirm document origin (download, email attachment) if you ingest email telemetry.
- Check if child process wrote files in user profile / temp directories.
- Hunt for persistence artifacts after execution (see section 14.7).


## 14.5 Ransomware early indicators (shadow copy deletion attempts)

### What to collect
- Process telemetry: vssadmin/wmic/bcdedit usage
- Windows Event Logs / Sysmon, EDR telemetry


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:process and process.name:(vssadmin.exe or wmic.exe or bcdedit.exe) and process.command_line:(*delete*shadow* or *shadowcopy* or *recoveryenabled* or *bootstatuspolicy*)


### Example EQL


In [ ]:
# EQL (use EQL search against the relevant index)
sequence by host.id with maxspan=15m
  [ process where process.name in ("vssadmin.exe","wmic.exe","bcdedit.exe") ]
  [ process where process.name in ("powershell.exe","cmd.exe","rundll32.exe") ]


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "process" AND process.name IN ("vssadmin.exe","wmic.exe","bcdedit.exe")
| WHERE process.command_line LIKE "*shadow*" OR process.command_line LIKE "*recoveryenabled*" OR process.command_line LIKE "*bootstatuspolicy*"
| KEEP @timestamp, host.name, user.name, process.name, process.command_line
| SORT @timestamp DESC
| LIMIT 200


### Triage tips
- Contain quickly: isolate host if you have high confidence.
- Check for lateral movement (SMB/RDP) and unusual admin tool usage.
- Look for spikes in file rename/write events and new extensions.


## 14.6 Lateral movement via remote services (RDP/SMB/WinRM)

### What to collect
- Network connection logs or EDR network events
- Windows logon events (success, logon type), RDP logs if available
- Fields: `source.ip`, `destination.ip`, `user.name`, `event.action`, `network.transport`, `destination.port`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:(authentication or network) and (destination.port:(3389 or 445 or 5985 or 5986) or network.protocol:(rdp or smb or winrm)) and event.outcome:success


### Example EQL


In [ ]:
# EQL (use EQL search against the relevant index)
sequence by user.name with maxspan=20m
  [ authentication where event.outcome == "success" ]
  [ network where destination.port in (3389, 445, 5985, 5986) ]


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE (event.category == "network" OR event.category == "authentication")
| WHERE destination.port IN (3389,445,5985,5986) OR network.protocol IN ("rdp","smb","winrm")
| STATS hits=COUNT(*) BY user.name, source.ip, destination.ip, destination.port
| SORT hits DESC
| LIMIT 100


### Triage tips
- Check whether `user.name` is expected to access the destination host.
- Look for 'new' admin shares usage (C$, ADMIN$) and remote service creation.
- Correlate with process creation on the destination host shortly after the logon.


## 14.7 Persistence (scheduled tasks / services / registry run keys)

### What to collect
- Windows scheduled task logs, service creation logs, Sysmon registry events
- Process telemetry for schtasks/sc.exe/reg.exe usage
- Fields: `process.command_line`, `registry.path`, `service.name`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:process and process.name:(schtasks.exe or sc.exe or reg.exe) and process.command_line:(*create* or *RunOnce* or *\\Services\\*)


### Example EQL


In [ ]:
# EQL (use EQL search against the relevant index)
sequence by host.id with maxspan=15m
  [ process where process.name in ("schtasks.exe","sc.exe","reg.exe") ]
  [ process where process.name in ("powershell.exe","cmd.exe","wscript.exe") ]


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "process" AND process.name IN ("schtasks.exe","sc.exe","reg.exe")
| WHERE process.command_line LIKE "*create*" OR process.command_line LIKE "*RunOnce*" OR process.command_line LIKE "*\\Services\\*"
| KEEP @timestamp, host.name, user.name, process.name, process.command_line
| SORT @timestamp DESC
| LIMIT 200


### Triage tips
- Validate the created task/service details; check path and signer of referenced binaries.
- Look for persistence under user-writable directories.
- Hunt for follow-on C2 connections from the persisted process.


## 14.8 Privilege escalation hints (admin group membership changes)

### What to collect
- Identity/audit logs for group membership changes
- Fields: `event.category:iam`, `event.action`, `user.name`, `user.target.*`, `group.name`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:iam and (event.action:*group* or event.type:*change*) and (group.name:(*Administrators* or *Domain Admins* or *Enterprise Admins*) or user.target.group.name:*)


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "iam"
| WHERE group.name LIKE "*Admin*" OR user.target.group.name LIKE "*Admin*"
| STATS changes=COUNT(*) BY user.name, user.target.name, group.name
| SORT changes DESC
| LIMIT 100


### Triage tips
- Check who performed the change (`user.name`) vs who was changed (`user.target.name`).
- Verify ticket/approval; look for changes outside change window.
- Correlate with new logons and sensitive access afterwards.


## 14.9 Data exfiltration (unusual outbound volume / rare destinations)

### What to collect
- Proxy/firewall logs, DNS logs, EDR network telemetry
- Fields: `destination.ip/domain`, `network.bytes`, `network.direction`, `user.name`, `host.name`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:network and network.direction:outbound and (destination.domain:* or destination.ip:*) and network.bytes:*


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "network" AND network.direction == "outbound"
| STATS total_bytes = SUM(network.bytes), events = COUNT(*) BY host.name, user.name, destination.domain
| SORT total_bytes DESC
| LIMIT 100


### Triage tips
- Compare against baselines (same host/user last 7/30 days).
- Check destination reputation / category (if enriched).
- Look for archiving tools (zip/rar/7z) and staging directories.


## 14.10 DNS-based suspicious activity (NXDOMAIN spikes / suspicious TLDs)

### What to collect
- DNS logs with `dns.question.name`, `dns.response_code`
- Fields: `host.name`, `source.ip`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:network and dns.question.name:* and (dns.response_code:(NXDOMAIN or SERVFAIL) or dns.question.name:(*.zip or *.top or *.xyz))


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "network" AND dns.question.name IS NOT NULL
| STATS n = COUNT(*) BY host.name, dns.response_code
| SORT n DESC
| LIMIT 200


### Triage tips
- Inspect query patterns: many unique domains/subdomains can indicate DGA.
- Check if the host also shows suspicious process activity.
- Consider adding TI enrichment for domains.


## 14.11 Web attacks (SQLi / traversal / scanning) in HTTP logs

### What to collect
- Web/WAF logs mapped to `url.*`, `http.*`
- Fields: `url.path`, `url.query`, `http.request.method`, `user_agent.original`, `source.ip`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:web and url.path:* and (url.query:(*union*select* or *../* or *%2e%2e%2f* or *or%201=1*) or url.path:(*../* or *%2e%2e%2f*))


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "web"
| WHERE url.query LIKE "*union*select*" OR url.query LIKE "*../*" OR url.query LIKE "*%2e%2e%2f*" OR url.query LIKE "*or%201=1*"
| STATS hits=COUNT(*) BY source.ip, url.path
| SORT hits DESC
| LIMIT 100


### Triage tips
- Review WAF action/outcome and whether requests were blocked.
- Check for 200 responses on suspicious payloads (possible successful exploitation).
- Correlate with new processes on the web server (if you have host telemetry).


## 14.12 Cloud suspicious activity (AWS example: unusual IAM changes)

### What to collect
- AWS CloudTrail mapped to ECS
- Fields: `event.category:iam`, `event.action`, `user.name`, `source.ip`, `cloud.region`


### Example KQL


In [ ]:
# KQL (paste into Kibana search bar)
event.category:iam and cloud.provider:aws and event.action:(CreateAccessKey or AttachUserPolicy or PutUserPolicy or CreateUser or UpdateAssumeRolePolicy or CreateLoginProfile)


### Example ES|QL


In [ ]:
# ES|QL (use Discover ES|QL or the ES|QL REST API)
FROM logs-*
| WHERE event.category == "iam" AND cloud.provider == "aws"
| WHERE event.action IN ("CreateAccessKey","AttachUserPolicy","PutUserPolicy","CreateUser","UpdateAssumeRolePolicy","CreateLoginProfile")
| KEEP @timestamp, user.name, source.ip, event.action, cloud.region
| SORT @timestamp DESC
| LIMIT 200


### Triage tips
- Confirm whether the actor is expected and whether an approved change exists.
- Check for concurrent unusual logins or API calls from new IPs.
- Rotate potentially compromised keys quickly.


# 15) Turning hunts into detections (rules)

Elastic Security supports rule types like:
- **Query rules** (KQL/Lucene/DSL depending on UI and version)
- **EQL rules** (sequence/correlation)
- **Indicator match** (TI matching)
- **Threshold** (count-based)
- **ML** (anomaly detection)

Key tuning knobs:
- Index patterns / data views used
- Lookback window and schedule interval
- Exceptions (host/user allow-lists)
- Risk score / severity mapping
- Actions (notifications, cases, response actions where supported)


## 16) Detections API (Kibana) — rule management (templates)

The Kibana Detections API can create/find/update detection rules.
This is useful for “detections-as-code” workflows.

> Exact request bodies depend on rule type and version. Use the Kibana API docs for your stack version.


In [ ]:

%%bash
# Template: find detection rules (Kibana API)
# curl -sS -X GET "$KIBANA_URL/api/detection_engine/rules/_find?per_page=20&page=1" \
#   -H "kbn-xsrf: true" \
#   -H "Authorization: ApiKey $ELASTIC_API_KEY"


In [ ]:

%%bash
# Template: create a simple query rule (Kibana API)
# NOTE: Rule schema evolves; consult Kibana "Create a detection rule" API docs for your version.

# curl -sS -X POST "$KIBANA_URL/api/detection_engine/rules" \
#   -H "kbn-xsrf: true" \
#   -H "Content-Type: application/json" \
#   -H "Authorization: ApiKey $ELASTIC_API_KEY" \
#   -d '{
#     "name": "Example - Suspicious PowerShell EncodedCommand",
#     "description": "Detect PowerShell with EncodedCommand",
#     "risk_score": 73,
#     "severity": "high",
#     "type": "query",
#     "query": "event.category:process and process.name:powershell.exe and process.command_line:*EncodedCommand*",
#     "index": ["logs-*"],
#     "from": "now-15m",
#     "interval": "5m",
#     "enabled": false
#   }'


# 17) Endpoint isolate/unisolate via Kibana API (templates)

Elastic provides Kibana API endpoints for endpoint isolation and release.
These are commonly used in SOAR automation.


In [ ]:

%%bash
# Template: isolate an endpoint (Kibana API)
# POST /api/endpoint/action/isolate
# Body typically includes endpoint IDs (agent IDs).

# curl -sS -X POST "$KIBANA_URL/api/endpoint/action/isolate" \
#   -H "kbn-xsrf: true" \
#   -H "Content-Type: application/json" \
#   -H "Authorization: ApiKey $ELASTIC_API_KEY" \
#   -d '{
#     "endpoint_ids": ["<endpoint_id_1>"],
#     "comment": "Isolate host due to suspected compromise"
#   }'


In [ ]:

%%bash
# Template: release an endpoint (Kibana API)
# POST /api/endpoint/action/unisolate

# curl -sS -X POST "$KIBANA_URL/api/endpoint/action/unisolate" \
#   -H "kbn-xsrf: true" \
#   -H "Content-Type: application/json" \
#   -H "Authorization: ApiKey $ELASTIC_API_KEY" \
#   -d '{
#     "endpoint_ids": ["<endpoint_id_1>"],
#     "comment": "Release after verification"
#   }'


# 18) Python client examples (Elasticsearch)

These examples show how to run:
- Query DSL searches
- EQL searches
- ES|QL queries

Install the official client:


In [ ]:

# !pip install elasticsearch>=8.0.0


In [ ]:

from elasticsearch import Elasticsearch

ELASTICSEARCH_URL = os.environ.get("ELASTICSEARCH_URL", "https://localhost:9200")
ELASTIC_API_KEY = os.environ.get("ELASTIC_API_KEY")  # base64 API key string

# Create client (verify_certs depends on your deployment)
es = Elasticsearch(
    ELASTICSEARCH_URL,
    api_key=ELASTIC_API_KEY,
    verify_certs=True,
)

# Basic info (will fail unless env vars point to a reachable cluster with valid creds)
# es.info()


## 18.1 Query DSL example (search recent suspicious PowerShell)

You can use Query DSL for full flexibility (bool filters, aggregations, etc.).


In [ ]:

query = {
    "size": 50,
    "sort": [{"@timestamp": "desc"}],
    "query": {
        "bool": {
            "filter": [
                {"term": {"event.category": "process"}},
                {"terms": {"process.name": ["powershell.exe", "pwsh.exe"]}},
                {"range": {"@timestamp": {"gte": "now-24h"}}},
            ],
            "should": [
                {"wildcard": {"process.command_line": "*EncodedCommand*"}},
                {"wildcard": {"process.command_line": "*Invoke-Expression*"}},
                {"wildcard": {"process.command_line": "*FromBase64String*"}},
            ],
            "minimum_should_match": 1,
        }
    },
}

# resp = es.search(index="logs-*", body=query)
# resp["hits"]["hits"][:2]


## 18.2 EQL example (sequence)

EQL is helpful for “A then B” patterns.


In [ ]:

eql_query = r'''
sequence by host.id with maxspan=10m
  [ process where process.name == "powershell.exe" and wildcard(process.command_line, "*EncodedCommand*") ]
  [ network where true ]
'''.strip()

# resp = es.eql.search(index="logs-*", query=eql_query, size=50)
# resp


## 18.3 ES|QL example (aggregate brute force failures)

ES|QL uses the `/_query` API under the hood.


In [ ]:

esql = r'''
FROM logs-*
| WHERE event.category == "authentication" AND event.outcome == "failure"
| STATS failures = COUNT(*) BY user.name, source.ip
| WHERE failures >= 20
| SORT failures DESC
| LIMIT 50
'''.strip()

# resp = es.esql.query(query=esql)
# resp


# 19) Hardening Elastic for security operations (high level)

Defenders should harden both the platform and their content:
- Enable security features (TLS, authN/authZ, API keys)
- Use least privilege roles; segment admin and analyst duties
- Restrict inbound access to Elasticsearch and Kibana
- Keep versions up to date; monitor CVEs
- Back up detection content (rules, exceptions, dashboards) as code
- Use ILM policies and storage tiers to control retention/cost


# 20) References (official docs & key resources)

Within Elastic’s ecosystem, these are especially useful:
- KQL reference
- EQL reference
- ES|QL reference + ES|QL REST API
- Elastic Security endpoint response actions
- Endpoint command reference (Elastic Endpoint CLI)
- Kibana API: detections + endpoint isolate/unisolate
- Public detection rules repository / explorer
- ECS reference

(Use Elastic Docs for your exact stack version.)
